## This is the script to make the coco json file for waymo dataset with our own annotation format. There are two kinds of coco anno json, one is for detection (bbox), therefore the json file only contains the bbox info of the objects in image. Another format is the segmentation-json file, which contains the bitmap info of the object contour. This bitmap can be represented in either straightward pixel coordinates or encoded as RLE. 

### This the to create the detection (bbox) json. 

In [ ]:
import os
import cv2
import json, yaml
import numpy as np
from PIL import Image
from collections import OrderedDict
from pycocotools import mask as cocomask
from pycocotools import coco as cocoapi
import matplotlib.pyplot as plt

In [ ]:
datapath = '/home/autolab/Data/waymo'

In [ ]:
info = {"year" : 2024,
        "version" : "1.0",
        "description" : "A coco json for Waymo Open Dataset of our own LiDAR-based annotation",
        "contributor" : "Gu Junyi (Claude)",
        "url" : "https://www.junyigu-claude.com",
        "date_created" : "13-03-2024"
            }
licenses = [{"id": 1,
             "name": "Attribution-NonCommercial",
             "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
                 }]
categories  = [{"id": 1,
                "isthing": 1,
                "name": "background",
                "supercategory": "background"
               },
              {"id": 2,
                "isthing": 1,
                "name": "vehicle",
                "supercategory": "vehicle" 
              },
              {"id": 3,
                "isthing": 1,
                "name": "human",
                "supercategory": "human"    
              },
              {"id": 4,
                "isthing": 1,
                "name": "other",
                "supercategory": "other"    
              }
             ]

In [ ]:
cat2id = {cat["name"]: catId+1 for catId, cat in enumerate(categories)}

In [ ]:
cat2id

In [ ]:
train_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/train_all.txt', dtype=str)
valid_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/early_stop_valid.txt', dtype=str)

In [ ]:
train_img_list

In [ ]:
valid_img_list

In [ ]:
def waymo_anno_class_relabel(annotation, category):
    """
    Reassign the indices of the objects in annotation(PointCloud);
    :parameter annotation: 0->ignore, 1->vehicle, 2->pedestrian, 3->sign,
                            4->cyclist, 5->background
    :return annotation: 0->background+sign, 1->vehicle
                            2->pedestrian+cyclist, 3->ingore
    """
    annotation = np.array(annotation)
    mask_ignore = annotation == 0
    mask_sign = annotation == 3
    mask_cyclist = annotation == 4
    mask_background = annotation == 5

    annotation[mask_sign] = 0
    annotation[mask_background] = 0
    annotation[mask_cyclist] = 2
    annotation[mask_ignore] = 3

    if category == 'vehicle':
        mask_rest = annotation != 1
        annotation[mask_rest] = 0
    elif category == 'human':
        mask_rest = annotation != 2
        annotation[mask_rest] = 0
    else:
        raise Exception('catrgory must be vehicle or human!')

    return annotation

In [ ]:
def det_mode_get_image_annotation_pairs(image_list):
        images = []
        annotations = []
        for imId, paths in enumerate(image_list):
            segments_info = []
            rgb_path = paths
            anno_path = paths.replace('/camera', '/annotation_relabel_rgb')
            print('Processing image of', rgb_path)
            rgb_name = rgb_path.split('/')[-1].split('.')[0]
            anno_name = anno_path.split('/')[-1].split('.')[0]
            assert (rgb_name == anno_name)
        
            rgb_img = np.array(Image.open(os.path.join(datapath + '/' + rgb_path)).convert('RGB'))
            anno_img = np.array(Image.open(os.path.join(datapath + '/' + anno_path)).convert('L'))  # Here open 3-channel anno in 1-channel
            if np.all(anno_img == 0):
                continue
                
            #rgb = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
            for c in ['vehicle', 'human']:
                category_id, bbox, area = det_mode_annotation_process(c, anno_img, rgb_img)
               # cv2.rectangle(rgb,(bbox[0],bbox[1]),(bbox[0]+bbox[2],bbox[2]+bbox[3]), (255,0,0), 1)
                if area != 0:
                    segments_info.append({"area": area,
                          "bbox": bbox,
                          "category_id": category_id,
                          "id": imId+1,
                         "iscrowd": 0})
            #cv2.imwrite(os.path.join('./test_img/' + rgb_name + '.png'), rgb)
            images.append({"date_captured" : "2024",
                           "file_name" : rgb_name + '.png', 
                           "id" : imId+1,
                           "license" : 1,
                           "url" : "",
                           "height" : 320,
                           "width" : 480})

            annotations.append({"file_name": anno_name + '.png',
                                "image_id": imId+1,
                                "segments_info": segments_info})
        return images, annotations

In [ ]:
def det_mode_annotation_process(category, anno, rgb=None):
    anno_relabel = waymo_anno_class_relabel(anno, category)
    area = np.count_nonzero(anno_relabel)
    [x, y, w, h] = cv2.boundingRect(anno_relabel)
        
    if category == 'vehicle':
        cat_id = 2
    elif category == 'human':
        cat_id = 3
    else:
         raise Exception('catrgory must be vehicle or human!')
    return cat_id, [x, y, w, h], area

In [ ]:
train_img_json_det, train_anno_json_det = det_mode_get_image_annotation_pairs(train_img_list)
train_json_data_det ={"info": info,
                 "images": train_img_json_det,
                 "licenses": licenses,
                 "annotations": train_anno_json_det,
                 "categories": categories}

with open('./waymo_train_det.json', 'w') as jsonfile:
    json.dump(train_json_data_det, jsonfile, sort_keys=True, indent=2)

In [ ]:
valid_img_json_det, valid_anno_json_det = det_mode_get_image_annotation_pairs(valid_img_list)
valid_json_data_det ={"info": info,
                 "images": valid_img_json_det,
                 "licenses": licenses,
                 "annotations": valid_anno_json_det,
                 "categories": categories}

with open('./waymo_valid_det.json', 'w') as jsonfile:
    json.dump(valid_json_data_det, jsonfile, sort_keys=True, indent=2)

In [ ]:
test_day_fair_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_day_fair.txt', dtype=str)
test_day_fair_img_list

In [ ]:
test_day_rain_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_day_rain.txt', dtype=str)
test_day_rain_img_list

In [ ]:
test_night_fair_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_night_fair.txt', dtype=str)
test_night_fair_img_list

In [ ]:
test_night_rain_img_list = np.genfromtxt('/home/autolab/Data/waymo/splits_clft/test_night_rain.txt', dtype=str)
test_night_rain_img_list

In [ ]:
test_day_fair_img_json_det, test_day_fair_anno_json_det = det_mode_get_image_annotation_pairs(test_day_fair_img_list)
test_day_fair_json_data_det ={"info": info,
                 "images": test_day_fair_img_json_det,
                 "licenses": licenses,
                 "annotations": test_day_fair_anno_json_det,
                 "categories": categories}

with open('./waymo_test_day_fair_det.json', 'w') as jsonfile:
    json.dump(test_day_fair_json_data_det, jsonfile, sort_keys=True, indent=2)

In [ ]:
test_day_rain_img_json_det, test_day_rain_anno_json_det = det_mode_get_image_annotation_pairs(test_day_rain_img_list)
test_day_rain_json_data_det ={"info": info,
                 "images": test_day_rain_img_json_det,
                 "licenses": licenses,
                 "annotations": test_day_rain_anno_json_det,
                 "categories": categories}

with open('./waymo_test_day_rain_det.json', 'w') as jsonfile:
    json.dump(test_day_rain_json_data_det, jsonfile, sort_keys=True, indent=2)

In [ ]:
test_night_fair_img_json_det, test_night_fair_anno_json_det = det_mode_get_image_annotation_pairs(test_night_fair_img_list)
test_night_fair_json_data_det ={"info": info,
                 "images": test_night_fair_img_json_det,
                 "licenses": licenses,
                 "annotations": test_night_fair_anno_json_det,
                 "categories": categories}

with open('./waymo_test_night_fair_det.json', 'w') as jsonfile:
    json.dump(test_night_fair_json_data_det, jsonfile, sort_keys=True, indent=2)

In [ ]:
test_night_rain_img_json_det, test_night_rain_anno_json_det = det_mode_get_image_annotation_pairs(test_night_rain_img_list)
test_night_rain_json_data_det ={"info": info,
                 "images": test_night_rain_img_json_det,
                 "licenses": licenses,
                 "annotations": test_night_rain_anno_json_det,
                 "categories": categories}

with open('./waymo_test_night_rain_det.json', 'w') as jsonfile:
    json.dump(test_night_rain_json_data_det, jsonfile, sort_keys=True, indent=2)

### This is to create the segmentation (bitmap) json.

In [ ]:
def seg_mode_get_image_annotation_pairs(image_list):
        images = []
        annotations = []
        i = 1
        for imId, paths in enumerate(image_list):
            segments_info = []
            rgb_path = paths
            anno_path = paths.replace('/camera', '/annotation_relabel_rgb')
            print('Processing image of', rgb_path)
            rgb_name = rgb_path.split('/')[-1].split('.')[0]
            anno_name = anno_path.split('/')[-1].split('.')[0]
            assert (rgb_name == anno_name)
        
            rgb_img = np.array(Image.open(os.path.join(datapath + '/' + rgb_path)).convert('RGB'))
            anno_img = np.array(Image.open(os.path.join(datapath + '/' + anno_path)).convert('L'))  # Here open 3-channel anno in 1-channel
            if np.all(anno_img == 0):
                continue
                
            #rgb = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
            for c in ['vehicle', 'human']:
                rle, category_id, bbox, area = seg_mode_annotation_process(c, anno_img, rgb_img)
               # cv2.rectangle(rgb,(bbox[0],bbox[1]),(bbox[0]+bbox[2],bbox[2]+bbox[3]), (255,0,0), 1)
                if area != 0:
                    annotations.append({"area": area,
                                        "bbox": bbox,
                                        "category_id": category_id,
                                        "image_id": imId+1,
                                        "iscrowd": 0,
                                        "segmentation": rle,
                                        "id": i})
                    i += 1
                    
            #cv2.imwrite(os.path.join('./test_img/' + rgb_name + '.png'), rgb)
            images.append({"date_captured" : "2024",
                           "file_name" : rgb_name + '.png', 
                           "id" : imId+1,
                           "license" : 1,
                           "url" : "",
                           "height" : 320,
                           "width" : 480})
        return images, annotations

In [ ]:
def seg_mode_annotation_process(category, anno, rgb=None):
    anno_relabel = waymo_anno_class_relabel(anno, category)
    area = np.count_nonzero(anno_relabel)
    RLE = cocomask.encode(np.asfortranarray(anno_relabel))
    #print(1 in anno_relabel)
    RLE['counts'] = RLE['counts'].decode('utf8')
    [x, y, w, h] = cv2.boundingRect(anno_relabel)
        
    if category == 'vehicle':
        cat_id = 2
    elif category == 'human':
        cat_id = 3
    else:
         raise Exception('catrgory must be vehicle or human!')
    return RLE, cat_id, [x, y, w, h], area

In [ ]:
test_day_fair_img_json_seg, test_day_fair_anno_json_seg = seg_mode_get_image_annotation_pairs(test_day_fair_img_list)
test_day_fair_json_data_seg ={"info": info,
                 "images": test_day_fair_img_json_seg,
                 "licenses": licenses,
                 "annotations": test_day_fair_anno_json_seg,
                 "categories": categories}

with open('./waymo_test_day_fair_seg.json', 'w') as jsonfile:
    json.dump(test_day_fair_json_data_seg, jsonfile, sort_keys=True, indent=2)

In [ ]:
test_day_rain_img_json_seg, test_day_rain_anno_json_seg = seg_mode_get_image_annotation_pairs(test_day_rain_img_list)
test_day_rain_json_data_seg ={"info": info,
                 "images": test_day_rain_img_json_seg,
                 "licenses": licenses,
                 "annotations": test_day_rain_anno_json_seg,
                 "categories": categories}

with open('./waymo_test_day_rain_seg.json', 'w') as jsonfile:
    json.dump(test_day_rain_json_data_seg, jsonfile, sort_keys=True, indent=2)

In [ ]:
test_night_fair_img_json_seg, test_night_fair_anno_json_seg = seg_mode_get_image_annotation_pairs(test_night_fair_img_list)
test_night_fair_json_data_seg ={"info": info,
                 "images": test_night_fair_img_json_seg,
                 "licenses": licenses,
                 "annotations": test_night_fair_anno_json_seg,
                 "categories": categories}

with open('./waymo_test_night_fair_seg.json', 'w') as jsonfile:
    json.dump(test_night_fair_json_data_seg, jsonfile, sort_keys=True, indent=2)

In [ ]:
test_night_rain_img_json_seg, test_night_rain_anno_json_seg = seg_mode_get_image_annotation_pairs(test_night_rain_img_list)
test_night_rain_json_data_seg ={"info": info,
                 "images": test_night_rain_img_json_seg,
                 "licenses": licenses,
                 "annotations": test_night_rain_anno_json_seg,
                 "categories": categories}

with open('./waymo_test_night_rain_seg.json', 'w') as jsonfile:
    json.dump(test_night_rain_json_data_seg, jsonfile, sort_keys=True, indent=2)

In [ ]:
train_img_json_seg, train_anno_json_seg = seg_mode_get_image_annotation_pairs(train_img_list)
train_json_data_seg ={"info": info,
                 "images": train_img_json_seg,
                 "licenses": licenses,
                 "annotations": train_anno_json_seg,
                 "categories": categories}

with open('./waymo_train_seg.json', 'w') as jsonfile:
    json.dump(train_json_data_seg, jsonfile, sort_keys=True, indent=2)

In [ ]:
valid_img_json_seg, valid_anno_json_seg = seg_mode_get_image_annotation_pairs(valid_img_list)
valid_json_data_seg ={"info": info,
                 "images": valid_img_json_seg,
                 "licenses": licenses,
                 "annotations": valid_anno_json_seg,
                 "categories": categories}

with open('./waymo_valid_seg.json', 'w') as jsonfile:
    json.dump(valid_json_data_seg, jsonfile, sort_keys=True, indent=2)

### Here are a visual example of how detection-json-create part works for image

In [ ]:
anno_0 = np.array(Image.open('/home/autolab/Data/waymo/labeled/day/rain/annotation_relabel_rgb/segment-89454214745557131_3160_000_3180_000_with_camera_labels_0000000135.png').convert('L'))
anno_0_copy = np.array(Image.open('/home/autolab/Data/waymo/labeled/day/rain/annotation_relabel_rgb/segment-89454214745557131_3160_000_3180_000_with_camera_labels_0000000135.png'))

In [ ]:
anno_0_relable = waymo_anno_class_relabel(anno_0, 'human')
np.count_nonzero(anno_0_relable)

In [ ]:
[x, y, w, h] = cv2.boundingRect(anno_0_relable)
x, y ,w, h

In [ ]:
image = cv2.rectangle(anno_0_relable, (x, y),(x+w, y+h),(1,1,1), 2)

In [ ]:
plt.imshow(anno_0_relable)

In [ ]:
plt.imshow(image)

### Here are a visual example of how segmentation-json-create part works for image

In [ ]:
# These info copied from the created seg-json file for waymo. Just a random copy.
example = {
      "area": 604,
      "bbox": [
        107,
        166,
        187,
        41
      ],
      "category_id": 2,
      "id": 1,
      "image_id": 1,
      "iscrowd": 0,
      "segmentation": {
        "counts": "ZdQ11f94]FNZ9NjF4LNY90iF3NMY90eFN250MY93gFO1LZ90cF2131MNO`94bFMNO`94bFLc9O\\F022`92`FMN0_93`FM0010e90ZF010e90ZF001`92`FM0001`92`FM0001`95`FJ01`92`FM001O`91_F0000001`9O`F0000001c9O]F00001c9O]F00001c9O]F00001`9O`F000001O`91_F00000d90\\F00000d90\\F00000d90\\F00000d90\\F00000d90\\F0000000a90_F0000000a90_F030M1`9O`F021NOa90_F030M0a93bFMa90\\F030d90kc00X\\O30000001LUF1^d00_E0]c00a\\O3OO1000i[h0NYdWO1O0O20N100000Sn01jQO3O00020NLYFOl900020^O0PG0C0Z90QGOE102e91[F0W9MSG3FOW9OSG7n8HUG0C0j90VF0m90R:0nE0N200UFNj90VF0j93000MVF0j90VF0j90VF0j90VF0j933Mkc00T\\O1NOVF0g93YFMg93YFMh9OXF100j90VF0j90VF0j933MVni1",
        "size": [
          320,
          480
        ]
      }
    }

In [ ]:
from pycocotools import coco, mask
def rle_to_bitmap(rle):
  bitmap = mask.decode(rle)
  return bitmap
mask_bitmap = rle_to_bitmap(example["segmentation"])

In [ ]:
img = Image.fromarray(mask_bitmap.astype(np.uint8) * 255, mode='L')
plt.imshow(img)